In [1]:
import os
import metaknowledge as mk
import networkx as nx
import pandas as pd
import csv
import math
import numpy as np
import statistics as stat
import matplotlib
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import rgb2hex
import gender_guesser.detector as gender
import plotly as py
from plotly.graph_objs import *
import numpy as np

os.chdir("C:/Users/Michael/PycharmProjects/DataIncubator/demo")
os.getcwd()

'C:\\Users\\Michael\\PycharmProjects\\DataIncubator\\demo'

In [2]:
#this notebook will take some the following files and generate a Plot.ly graph

#    bioactive.txt : Web of Science (WoS) Record Collection, the data for one group
#    CosCiting.csv : WoS subject categories distance matrix. from Rafols, Porter, Leydesdorff 2010
#    DisciplinesList3.csv : WoS subject categories by 6 factors from Radols, Porter, Leydesdorff
#    WoS History Nov 2017 : As of last fall, every journal index by WoS and its category
#    tags.csv : Tags file for bioactive network

Cosciting = pd.read_csv('CosCiting.csv', sep=',', header=0, index_col=0)
DisList = pd.read_csv('DisciplinesList3.csv', sep=',',header=0, index_col='Discipline')
JournalCats = pd.read_excel('WoS History Nov 2017.xlsx', header=0)
JournalCats.set_index('20 Char', inplace=True)
tags = pd.read_csv('Tags.csv', header=0, index_col='Name')
RC = mk.RecordCollection("bioactive.txt")

print("data loaded")

data loaded


In [4]:
G = nx.Graph()
authors ={}

for R in RC:
    for author1 in R['AF']:
        if author1 not in G.nodes() :
            G.add_node(author1, Papers = 1)
        else :
            G.nodes[author1]['Papers'] += 1
        authors[author1] = []

for R in RC:
    for author in R['AF']:
        try:
            authors[author]=authors[author]+R['WC']
        except:
            KeyError

for R in RC:
    for author1 in R['AF'] :
        for author2 in R['AF']:
            if author1 != author2 :
                if (author1, author2) not in G.edges():
                    G.add_edge(author1, author2, Weight = 0.5)
                else :
                    G.edges[author1, author2]['Weight'] += 0.5

print("networks constructed")

networks constructed


In [5]:
#this should convert authors into a dictionary of subject categories with counts
for auth in authors :
    line = authors[auth]
    dic = {}
    for item in line :
        if item.upper() in dic :
            dic[item.upper()] = dic[item.upper()] + 1
        else :
            dic[item.upper()] = 1
    authors[auth] = dic

print("discipline lines created")

discipline lines created


In [ ]:
#okay, let's calculate the Gini coefficient and Rao-Stirling Index
for auth in authors :
    sample = authors[auth] #authors[auth] is the numbered dictionary of WoS categories an author has contributed to
    total = 0
    for sc in sample :
        total = total + sample[sc]
    gini = 0
    for a in sample :
        for b in sample :
            if a != b :
                gini = gini + (sample[a]/total)*(sample[b]/total)
    G.nodes[auth]['Gini'] = gini



In [21]:
for auth in authors :
    sample  = authors[auth].copy()
    samplelist = list(sample)
    total = 0
    for sc in sample :
        total = total + sample[sc]
    SDI = 0
    i = 0
    j = 1
    while i < len(samplelist) :
        j = i + 1
        while j < len(samplelist) :
            WCi = samplelist[i]
            WCj = samplelist[j]
            try :
                invdistance = float(Cosciting.loc[WCi.upper(),WCj.upper()])
            except KeyError :
                invdistance = 1
            SDI = SDI + (1 - invdistance)*(sample[WCi]/total)*(sample[WCj]/total)
            j =j +1
        i =i + 1
    G.nodes[auth]['Rao-Stirling']= SDI
    
print("Rao-Stirling index calculated")


Rao-Stirling index calculated


In [22]:
#assign most common discipline to authors

for auth in authors:
    sample  = authors[auth].copy()
    try:
        discipline = max(sample, key=sample.get)
    except :
        discipline ='ERROR'
    G.nodes[auth]['Discipline'] = discipline

print ("authors associated with disciplines")

authors associated with disciplines


In [23]:
#this assigns each Discipline a distinct color by the six primary and secondary colors
#lightness is along the factor identified by Rafols, Porter, Leydesdorff 2010
DisList = pd.read_csv('DisciplinesList4.csv', sep=',',header=0, index_col='Discipline')

Fields = {'physical sciences':[], 'social sciences':[], 'computer sciences':[], 'field sciences':[],'life sciences':[], 'behavioral sciences':[]}
for row in DisList.itertuples():
    item = row[0]
    try:
        Field = DisList.at[item.upper(), 'Field']
        Fields[Field] = Fields[Field]+[(item,1)]
    except :
        pass
###OLD Colors
#LifeSciencesColors = LinearSegmentedColormap.from_list('natural', ['#7ca5ff','#0f2656'], N=100)
#PhysicalSciencesColors = LinearSegmentedColormap.from_list('natural', ['#edb568','#911414'], N=100)
#FieldColors = LinearSegmentedColormap.from_list('life', ['#87db91','#004908'], N=100)
#BehavioralColors = LinearSegmentedColormap.from_list('social', ['#d293ff','#421166'], N=100)
#SocialSciencesColors = LinearSegmentedColormap.from_list('social', ['#f959f9','#7c153e'], N=100)
#CSColors = LinearSegmentedColormap.from_list('engineering', ['#ffe856','#4f4400'], N=100)

LifeSciencesColors = LinearSegmentedColormap.from_list('life', ['#90d7fe','#010b6b'], N=100)
PhysicalSciencesColors = LinearSegmentedColormap.from_list('physics', ['#ffc675','#7a3700'], N=100)
FieldColors = LinearSegmentedColormap.from_list('field', ['#94ff70','#004908'], N=100)
BehavioralColors = LinearSegmentedColormap.from_list('behave', ['#fea4fa','#57135e'], N=100)
SocialSciencesColors = LinearSegmentedColormap.from_list('social', ['#ff817a','#741128'], N=100)
CSColors = LinearSegmentedColormap.from_list('engineering', ['#ebe77f','#7a7000'], N=100)

for field in Fields :
    if field == 'life sciences' :
        cm = LifeSciencesColors
    if field == 'physical sciences' :
        cm = PhysicalSciencesColors
    if field == 'field sciences' :
        cm = FieldColors
    if field == 'behavioral sciences' :
        cm = BehavioralColors
    if field == 'social sciences' :
        cm = SocialSciencesColors
    if field == 'computer sciences' :
        cm = CSColors
    i =0
    while i < len(Fields[field]):
        val = i/len(Fields[field])+.001
        color = rgb2hex(cm(val))
        Fields[field][i] = (Fields[field][i][0],Fields[field][i][1],field,color)
        i = i + 1

ColorList = []
for field in Fields:
    for item in Fields[field]:
        ColorList = ColorList +[item]

NewColors = pd.DataFrame.from_records(ColorList, columns = ['Discipline','Count','Field','Color'], index='Discipline')

for node in G.nodes():
    try:
        disc = G.nodes[node]['Discipline']
        color = NewColors.at[disc,'Color']
        G.nodes[node]['DisColor'] = color
    except :
        G.nodes[node]['DisColor'] = '#444'
        
print("colors generated for each discipline")

colors generated for each discipline


In [24]:
#and one more module to calculate Rao-Stirling Scores for Edges
for edge in G.edges():
    a = edge[0]
    b = edge[1]
    try:
        invdistance = float(Cosciting.loc[G.nodes[a]['Discipline'].upper(),G.nodes[b]['Discipline'].upper()])
    except KeyError:
        invdistance = 1
    SDI = 1 - invdistance
    G.edges[edge]['Rao-Stirling-Connection'] = SDI

print("Rao-Stirling distance calculated for edges")

Rao-Stirling distance calculated for edges


In [25]:
#create the Core network of people in the research group, and calculate Betweenness Centrality

for node in G.nodes():
    G.node[node]['Sample'] = 0

for row in tags.itertuples():
    try:
        G.nodes[row[0]]['Sample']=row[1]
        G.nodes[row[0]]['Status']=row[2]
    except:
        pass

subnodes = []
for node in G.nodes():
    if G.nodes[node]['Sample']== 1:
        subnodes.append(node)

Core = nx.subgraph(G, subnodes)
BtwCore = nx.betweenness_centrality(Core)

for name in BtwCore:
    G.nodes[name]['BtwCore'] =BtwCore[name]
    Core.nodes[name]['BtwCore'] =BtwCore[name]

for edge in G.edges():
    type = G.nodes[edge[0]]['Sample']+G.nodes[edge[1]]['Sample']
    G.edges[edge]['Type'] = type

print("Group members identified, network updated")

Group members identified, network updated


In [26]:
#check that everything is okay before we plot

for node in G.nodes():
    print (node, G.nodes[node]['Discipline'],G.nodes[node]['DisColor'],
           G.nodes[node]['Rao-Stirling'])


Miller, Anne-Frances BIOCHEMISTRY & MOLECULAR BIOLOGY #8cd1fa 0.2504090909090909
Pitsawong, Warintra BIOCHEMISTRY & MOLECULAR BIOLOGY #8cd1fa 0.036086419753086414
Haynes, Chad A. BIOCHEMISTRY & MOLECULAR BIOLOGY #8cd1fa 0.036086419753086414
Koder, Ronald, Jr. BIOCHEMISTRY & MOLECULAR BIOLOGY #8cd1fa 0.036086419753086414
Rodgers, David W. BIOCHEMISTRY & MOLECULAR BIOLOGY #8cd1fa 0.2205066666666667
Wattamwar, Paritosh P. ENGINEERING, BIOMEDICAL #395ba5 0.21403999999999998
Biswal, Dipti POLYMER SCIENCE #d59950 0.24811555555555556
Cochran, David B. ENGINEERING, BIOMEDICAL #395ba5 0.10016326530612243
Lyvers, Adam C. ENGINEERING, BIOMEDICAL #395ba5 0.0325
Eitel, Richard E. MATERIALS SCIENCE, CERAMICS #e7ac60 0.240944
Anderson, Kimberly W. MATERIALS SCIENCE, BIOMATERIALS #aa6b2b 0.2802234819853867
Hilt, J. Zach MATERIALS SCIENCE, BIOMATERIALS #aa6b2b 0.2682512894267013
Dziubla, Thomas D. MATERIALS SCIENCE, BIOMATERIALS #aa6b2b 0.2786627906976746
Fisher, Paul D. ENGINEERING, BIOMEDICAL #395ba5

 0.09425
Khosravi, Medhi SURGERY #0d1b77 0.09425
Tobias, Joseph D. SURGERY #0d1b77 0.09425
Hamorsky, Krystal Teasley CHEMISTRY, MULTIDISCIPLINARY #f2b869 0.1910277777777778
Browning, James F. CHEMISTRY, APPLIED #b97b38 0.1148125
Xul, Shifen CHEMISTRY, MULTIDISCIPLINARY #f2b869 0.07544000000000001
de Leon, Ame C. CHEMISTRY, MULTIDISCIPLINARY #f2b869 0.07544000000000001
Noinaj, Nicholas MULTIDISCIPLINARY SCIENCES #87cbf5 0.036250000000000004
Bhasin, Sonia K. MULTIDISCIPLINARY SCIENCES #87cbf5 0.036250000000000004
Scoggin, Kirsten E. MULTIDISCIPLINARY SCIENCES #87cbf5 0
Goldsmith, Jacob CHEMISTRY, PHYSICAL #f8bf6f 0.02433333333333333
Moore, Ida M. (Ki) NURSING #6a236f 0
Merkle, Carrie J. NURSING #6a236f 0
Byrne, Howard NURSING #6a236f 0
Ross, Adam NURSING #6a236f 0
Ameli, Sara S. NURSING #6a236f 0
Montgomery, David W. NURSING #6a236f 0
Wu, Pei-Jung CHEMISTRY, MULTIDISCIPLINARY #f2b869 0.05411111111111111
Arreaza, Roberto CHEMISTRY, MULTIDISCIPLINARY #f2b869 0.05411111111111111
Vasilakes, 

In [27]:
path =  'bioactive'

def tooltiptext(node):
    text = str( '<b>' +node +'</b><br>' +
         G.nodes[node]['Discipline'] + '<br>' +
         'Papers: ' + str(G.nodes[node]['Papers']) + '<br>' +
         'Rao-Stirling: ' + str(round(G.nodes[node]['Rao-Stirling'],4)) + '<br>'
         )
    return text

#this bit of code changes papers to whatever nmodesize we want
node_size2 = list(nx.get_node_attributes(G,'Papers').values())
node_size = []
for i in node_size2 :
    k = float(i)
    node_size.append(math.sqrt(k)*3)

#we're going to place the mark at std, and show 3 categories.
#lineweight 1
#lineweight 1-mark
#lineweight higher
weights = list(nx.get_edge_attributes(G, 'Weight').values())
a=[x for x in weights if x != 1]
mark = np.percentile(a, 90)

for edge in G.edges() :
    if G.edges()[edge]['Weight'] == 1 :
        G.edges()[edge]['weightCat'] = 1
    elif 1 < G.edges()[edge]['Weight'] < mark :
        G.edges()[edge]['weightCat'] = 2
    else :
        G.edges()[edge]['weightCat'] = 3

#this version draws our graph from the data. Now to get the formatting right...
pos=nx.spring_layout(G, weight='weight',iterations=250)

xs=[]
ys=[]

for item in pos:
    xs.append(pos[item][0])
    ys.append(pos[item][1])

ylim = 2*stat.stdev(ys)
xlim = 2*stat.stdev(xs)

for item in pos :
    if pos[item][0]>xlim :
        pos[item][0] = xlim
    if pos[item][0] < -xlim :
        pos[item][0] = -xlim
    if pos[item][1] > ylim :
        pos[item][1] = ylim
    if pos[item][1] < -ylim :
        pos[item][1] = -ylim

for node in G :
    G.nodes[node]['pos']=pos[node]
    
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

print('tooltip created')

'file://C:\\Users\\Michael\\PycharmProjects\\DataIncubator\\demo\\bioactive RSEdges_Disciplines.html'

In [31]:
#figure 1, Disciplinary

edge_traces = []

cm = matplotlib.cm.get_cmap(name='magma')
for edge in G.edges():
    width =.75
    color = rgb2hex(cm(G.edges[edge]['Rao-Stirling-Connection']))

    edge_trace = Scatter(
        x=[],
        y=[],
        text=[],
        line=dict(width=width, color=str(color)),
        hoverinfo='text',
        mode='lines')

    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

    edge_traces = edge_traces + [edge_trace]

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        color=list(nx.get_node_attributes(G,'DisColor').values()),
        opacity=1,
        size= node_size,
        line= dict(width=list(nx.get_node_attributes(G,'Sample').values()),color='#000')
    )
)

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

for node in G :
    node_info = tooltiptext(node)
    node_trace['text'].append(node_info)

fig = Figure(data=Data(edge_traces + [node_trace]),
             layout=Layout(
                title=str(path + ' Rao-Stirling Edge Network'),
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.offline.plot(fig, filename=str(path + ' RSEdges_Disciplines.html'), image_height=1000, image_width=1000)
#py.plotly.plot(fig, filename=str(path + ' CompleteDisciplines.html'), image_height=1000, image_width=1000)


'file://C:\\Users\\Michael\\PycharmProjects\\DataIncubator\\demo\\bioactive RSEdges_Disciplines.html'

In [ ]:
#figure2 Rao-Stirling

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YlOrRd',
        reversescale=True,
        color=[],
        size=node_size,
        opacity =1,
        colorbar=dict(
            thickness=15,
            title='Rao-Sterling Diversity',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=list(nx.get_node_attributes(G,'Sample').values()),color='#000')
    ))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

for node in G :
    node_trace['marker']['color'].append(G.nodes()[node]['Rao-Stirling'])
    node_info = tooltiptext(node)
    node_trace['text'].append(node_info)

fig2 = Figure(data=Data(edge_traces + [node_trace]),
             layout=Layout(
                title=str(path + ' Complete Rao-Sterling Network'),
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.offline.plot(fig2, filename=str(path + ' CompleteRS.html'), image_height=1000, image_width=1000)

'file://C:\\Users\\Michael\\PycharmProjects\\DataIncubator\\demo\\bioactive RSEdges_Disciplines.html'